In [ ]:
# LINE, SKU, DESCRIPTION LINE ITEM COMMENTS, UNIT COST/ RETAIL PRICE, QTY, ITEM TOTAL

In [60]:
import pdfplumber
import pandas as pd
import numpy as np
import re

In [78]:
file_path = "static/de.pdf"
with pdfplumber.open(file_path) as pdf:
    # table = pdf.pages[0]
    table = pdf.pages[0].extract_tables()
    # table_n = pdf.pages[0].find_tables()
    text = pdf.pages[0].extract_text(layout=False)
    # table = pdf.pages[0].extract_table(table_settings={
    #                                                     "vertical_strategy": "text",
    #                                                     "horizontal_strategy": "explicit",
    #                                                     "explicit_horizontal_lines": [50,50,50,50],
    #                                                     "text_x_tolerance": 10,
    #                                                    })
# tables

In [ ]:
print(text, sep="\t")

In [79]:
table.sort(key=len)
table = table[-2:]

In [80]:
if table[0][0][0] == "LINE":
    line_table = table[0]
    other_table = table[1]
else:
    line_table = table[1]
    other_table = table[0]

## Line Item Part

In [81]:
data_frame = pd.DataFrame(line_table)
data_frame.columns = data_frame.iloc[0]
data_frame = data_frame.drop(0)

In [82]:
data_frame

,LINE,SKU,VENDOR\nPN,UPC/GTIN,DESCRIPTION LINE ITEM\nCOMMENTS,MARKS AND\nNUMBERS,UNIT COST/\nRETAIL\nPRICE,QTY,UOM,ITEM\nTOTAL
1,1,3040446,,,Item Detail: INNO ORGANIC\nCOCONUT CLUSTERS 50...,,Unit Price: 9.51,16320.0,Each,"155,203.20"
2,1,# of Line\nItems,Merchandise Total,None,None,None,None,16320,,"155,203.20"


In [83]:
data_frame.columns = data_frame.columns.str.replace('\n', ' ')
required_columns = ['LINE', 'SKU', 'DESCRIPTION LINE ITEM COMMENTS', 'UNIT COST/ RETAIL PRICE', 'QTY', 'ITEM TOTAL']
required_columns = list(map(lambda text: re.sub(r'\s+', ' ', text.strip()), required_columns))

column_name = ['LINE']
required_columns = ['LINE', 'SKU', 'DESCRIPTION LINE ITEM COMMENTS', 'UNIT COST/ RETAIL PRICE', 'QTY', 'ITEM TOTAL']
required_columns.extend(column_name)
required_columns = np.unique(required_columns)
selected_df = data_frame[required_columns].copy()

selected_df = selected_df.iloc[:-1]
selected_df = selected_df.replace('\n',' ', regex=True)
final_df = selected_df.set_index("LINE")

In [84]:
final_df

,DESCRIPTION LINE ITEM COMMENTS,ITEM TOTAL,QTY,SKU,UNIT COST/ RETAIL PRICE
LINE,,,,,
1,Item Detail: INNO ORGANIC COCONUT CLUSTERS 500...,"155,203.20",16320.0,3040446,Unit Price: 9.51


In [85]:
json_output = final_df.to_json(orient = 'columns')
json_output

'{"DESCRIPTION LINE ITEM COMMENTS":{"1":"Item Detail: INNO ORGANIC COCONUT CLUSTERS 500G P272"},"ITEM TOTAL":{"1":"155,203.20"},"QTY":{"1":"16320.0"},"SKU":{"1":"3040446"},"UNIT COST\\/ RETAIL PRICE":{"1":"Unit Price: 9.51"}}'

In [99]:
selected_df.to_csv("output.csv")

## Other Parts 

In [86]:
from itertools import chain
import re

In [87]:
vendor_name = "Costco Canada"
result = list(chain(other_table[0], other_table[1], other_table[3]))
order_number_pattern = r'Order #:\s+(\d+)'
match = re.search(order_number_pattern, text)
order_num = match.group(1)

In [37]:
result

['PO Date:\n08/16/2022',
 'Requested Delivery Date:\n08/25/2022',
 'Requested Ship Date:\n08/24/2022',
 'Cancel Date:\n08/29/2022',
 'Vendor #:\n0007764600',
 'Department #:\n13COSTCO DEPARTMENT']

In [88]:
# result = result.replace('\n',' ', regex=True)
result = list(map(lambda text: text.replace('\n', ''), result))

In [39]:
result

['PO Date:08/16/2022',
 'Requested Delivery Date:08/25/2022',
 'Requested Ship Date:08/24/2022',
 'Cancel Date:08/29/2022',
 'Vendor #:0007764600',
 'Department #:13COSTCO DEPARTMENT']

In [89]:
dic_all_item = {}
pattern_a = r":(.+)"
pattern_b = r"(.+):"
for itter in result:
    match_bef = re.search(pattern_b, itter)
    match_aft = re.search(pattern_a, itter)
    dic_all_item[match_bef.group(1)] = match_aft.group(1)

In [90]:
dic_all_item

{'PO Date': '07/05/2023',
 'Requested Delivery Date': '07/20/2023',
 'Requested Ship Date': '07/14/2023',
 'Cancel Date': '07/14/2023',
 'Vendor #': '0007764600',
 'Department #': '12COSTCO DEPARTMENT'}

In [91]:
list_other = ["PO Date", "Requested Ship Date", "Requested Delivery Date", "Cancel Date", "Vendor #"]

In [92]:
final_dic_other = {}
final_dic_other['Vendor Name'] = vendor_name
final_dic_other['Order Number'] = order_num
for zzz in dic_all_item:
    if zzz in list_other:
        final_dic_other[zzz] = dic_all_item[zzz]

In [93]:
final_dic_other

{'Vendor Name': 'Costco Canada',
 'Order Number': '005710705035',
 'PO Date': '07/05/2023',
 'Requested Delivery Date': '07/20/2023',
 'Requested Ship Date': '07/14/2023',
 'Cancel Date': '07/14/2023',
 'Vendor #': '0007764600'}

In [59]:
final_dic_other
print(required_columns)

['DESCRIPTION LINE ITEM COMMENTS' 'ITEM TOTAL' 'LINE' 'QTY' 'SKU'
 'UNIT COST/ RETAIL PRICE']
